# Prepare the marine ecoregion / pelagic province data


### Import libraries and set up environment

In [2]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [3]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [ ]:
# Make sure Caribbean_detailed_subregions_mol is in the project (made in EC_study_area_prep)

In [4]:
# Add the raw marine ecoregion layers from the O drive to your ArcGIS project

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.conversion.FeatureClassToGeodatabase(['O:/f00_data/WCMC-036-MEOW-PPOW-2007-2012/DataPack-14_001_WCMC036_MEOW_PPOW_2007_2012_v1/01_Data/WCMC-036-MEOW-PPOW-2007-2012-NoCoast.shp'],
                                           output_gdb)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb'>

In [8]:
# Reproject the ecoregion data to Mollweide

in_feature = "WCMC_036_MEOW_PPOW_2007_2012_NoCoast"
out_feature = "MEco_mol"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.management.Project(in_feature, out_feature, 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
    

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MEco_mol'>

In [9]:
# Clip ecoregion data to study area

in_feature = "MEco_mol"
clip_feature = "Caribbean_detailed_subregions_mol"
out_feature = "\\MEco_mol_Carib"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.Clip(in_feature, clip_feature, out_feature)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\\\MEco_mol_Carib'>

In [14]:
# Make a field to identify which ecoregion/pelagic province will be used
# ecoregion should ideally be used if there is one, otherwise use pelagic province

arcpy.management.CalculateField(
    in_table="MEco_mol_Carib",
    field="MEOW_PPOW",
    expression="MEOW_PPOW(!ECOREGION!, !PROVINC!)",
    expression_type="PYTHON3",
    code_block="""def MEOW_PPOW(ECOREGION, PROVINC):
    if ECOREGION.strip() != "":
        return ECOREGION
    else:
        return PROVINC""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MEco_mol_Carib'>